<a href="https://colab.research.google.com/github/Madhuanabala/breast-cancer/blob/main/to_convert_to_binary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
import pandas as pd
input_file = '/content/RDkit_descriptors.csv'

!pip install scikit-learn
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
data = pd.read_csv(input_file)
data

,Name,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,chunk_index
0,CHEMBL266703,15.351667,-5.897321,15.351667,1.284540,0.007708,1179.286,1108.726,1178.473597,452,...,0,0,0,0,0,0,0,0,0,0
1,CHEMBL2145445,14.779065,-5.160822,14.779065,0.750052,0.044552,917.021,860.573,916.378240,352,...,0,0,0,0,0,0,0,0,0,0
2,CHEMBL266185,15.352054,-5.890709,15.352054,1.278628,0.009166,1135.233,1068.705,1134.447382,434,...,0,0,0,0,0,0,0,0,0,0
3,CHEMBL19195,8.888157,-4.013815,8.888157,0.750052,0.428412,373.496,346.280,373.204179,144,...,0,0,0,0,0,0,0,0,0,0
4,CHEMBL83,8.907225,-4.200682,8.907225,0.874632,0.450573,371.524,342.292,371.224915,144,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1812,CHEMBL5440580,14.085560,-4.561672,14.085560,0.161944,0.800120,308.381,288.221,308.152478,118,...,0,0,0,0,0,0,0,0,0,0
1813,CHEMBL5417545,14.103701,-4.533282,14.103701,0.126389,0.783301,342.826,323.674,342.113505,124,...,0,0,0,0,0,0,0,0,0,0
1814,CHEMBL225542,14.366070,-1.471026,14.366070,0.038746,0.564813,322.303,311.215,322.097837,118,...,0,0,0,0,0,0,0,0,0,0
1815,CHEMBL1093100,8.508773,-2.948503,8.508773,0.007460,0.522767,321.409,306.289,321.104817,114,...,0,0,0,0,0,0,0,0,0,0


In [57]:
if 'Unnamed: 0' in data.columns:
    data = data.drop(columns=['Unnamed: 0'])

In [58]:
def discretize_column(col, bins=3):
    """Discretize a numerical column into equal-width bins."""
    return pd.cut(col, bins=bins, labels=False)

In [59]:
discretized_data = data.copy()
for column in discretized_data.select_dtypes(include=['float64', 'int64']).columns:
    discretized_data[column] = discretize_column(discretized_data[column])

In [60]:
encoder = OneHotEncoder(sparse_output=False)
encoded_data = encoder.fit_transform(discretized_data)

In [61]:
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(discretized_data.columns))

In [62]:
output_file = "onehot_encoded_rdkit_descriptors.csv"
encoded_df.to_csv(output_file, index=False)

print(f"One-hot encoded data saved to {output_file}")

One-hot encoded data saved to onehot_encoded_rdkit_descriptors.csv


In [65]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Load the CSV file
input_file = '/content/RDkit_descriptors.csv'
data = pd.read_csv(input_file)

# Drop the 'Unnamed: 0' column if it exists
if 'Unnamed: 0' in data.columns:
    data = data.drop(columns=['Unnamed: 0'])

# Separate the 'Name' column to keep it unchanged in the output
name_column = data['Name']

# Drop the 'Name' column for encoding
data_to_encode = data.drop(columns=['Name'])

# Function to discretize numerical columns into bins
def discretize_column(col, bins=3):
    """Discretize a numerical column into equal-width bins."""
    return pd.cut(col, bins=bins, labels=False)

# Create a copy of the data to apply discretization
discretized_data = data_to_encode.copy()

# Discretize the numerical columns (float64 or int64)
for column in discretized_data.select_dtypes(include=['float64', 'int64']).columns:
    discretized_data[column] = discretize_column(discretized_data[column])

# Initialize the OneHotEncoder
encoder = OneHotEncoder(sparse_output=False, drop='if_binary')  # Avoid extra columns by handling binary variables

# Fit and transform the data to binary (one-hot encoding)
encoded_data = encoder.fit_transform(discretized_data)

# Convert the encoded data to a DataFrame
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(discretized_data.columns))
encoded_df = encoded_df.iloc[:, :209]
# Add the 'Name' column back to the encoded DataFrame
encoded_df.insert(0, 'Name', name_column)

# Save the encoded data to a new CSV file
output_file = "onehot_encoded_rdkit_descriptors.csv"
encoded_df.to_csv(output_file, index=False)

print(f"One-hot encoded data saved to {output_file}")


One-hot encoded data saved to onehot_encoded_rdkit_descriptors.csv
